#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
*   https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
*    https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)

##P0. importar librerias

In [ ]:
!pip install tensorflow-gpu

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time
import sys

uso de GPU para entrenar en tensorflow

---
https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa


print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

In [2]:
print("Version: ", tf.__version__)
#print("Eager mode: ", tf.executing_eagerly())
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta NOT AVAILABLE
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
tf.device('/gpu:0') #activando la CPU
#tf.device('/device:GPU:0') #activando la GPU 

##P0. Descarga y preprocesado de los datos

In [4]:
fileDL= tf.keras.utils.get_file('arte.txt','https://raw.githubusercontent.com/raleo-pin/deep/main/segundo_corte/data/arte.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

98304/87055 [=================================] - 0s 0us/step


In [5]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

el arte de la guerra capitulo 1 sobre la evaluacion sun tzu dice la guerra es de vital importancia para el estado es el dominio de la vida o de la muerte el camino hacia la supervivencia o la perdida del imperio es forzoso manejarla bien no reflexionar seriamente sobre todo lo que le concierne es dar prueba de una culpable indiferencia en lo que respecta a la conservacion o perdida de lo que nos es mas querido y ello no debe ocurrir entre nosotros hay que valorarla en terminos de cinco factores fundamentales y hacer comparaciones entre diversas condiciones de los bandos rivales con vistas a determinar el resultado de la guerra el primero de estos factores es la doctrina el segundo el tiempo el tercero el terreno el cuarto el mando y el quinto la disciplina la doctrina significa aquello que hace que el pueblo este en armonia con su gobernante de modo que le siga donde sea sin temer por sus vidas ni a correr cualquier peligro el tiempo significa el ying y el yang la noche y el dia el fri

##P1. entendiendo el texto

In [6]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:83333 caracteres
el texto esta compuesto de estos :36 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'el arte de la guerra capitulo 1 sobre la evaluacion sun tzu dice la guerra es de vital importancia p'
array([15, 22,  0, 11, 28, 30, 15,  0, 14, 15,  0, 22, 11,  0, 17, 31, 15,
       28, 28, 11,  0, 13, 11, 26, 19, 30, 31, 22, 25,  0,  2,  0, 29, 25,
       12, 28, 15,  0, 22, 11,  0, 15, 32, 11, 22, 31, 11, 13, 19, 25, 24,
        0, 29, 31, 24,  0, 30, 35, 31,  0, 14, 19, 13, 15,  0, 22, 11,  0,
       17, 31, 15, 28, 28, 11,  0, 15, 29,  0, 14, 15,  0, 32, 19, 30, 11,
       22,  0, 19, 23, 26, 25, 28, 30, 11, 24, 13, 19, 11,  0, 26])


In [8]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

##P3. preparar los datos para ser usados en la RNN

In [9]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'el arte de la guerra capitulo 1 sobre la evaluacion sun tzu dice la guerra es de vital importancia para el estado es el dominio de la vida o de la muer'
'te el camino hacia la supervivencia o la perdida del imperio es forzoso manejarla bien no reflexionar seriamente sobre todo lo que le concierne es dar '
'prueba de una culpable indiferencia en lo que respecta a la conservacion o perdida de lo que nos es mas querido y ello no debe ocurrir entre nosotros h'
'ay que valorarla en terminos de cinco factores fundamentales y hacer comparaciones entre diversas condiciones de los bandos rivales con vistas a determ'
'inar el resultado de la guerra el primero de estos factores es la doctrina el segundo el tiempo el tercero el terreno el cuarto el mando y el quinto la'
' disciplina la doctrina significa aquello que hace que el pueblo este en armonia con su gobernante de modo que le siga donde sea sin temer por sus vida'
's ni a correr cualquier peligro el tiempo significa el ying y el yang la no

###P3.1 separar los datos en agrupamientos (batches)

In [10]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'el arte de la guerra capitulo 1 sobre la evaluacion sun tzu dice la guerra es de vital importancia para el estado es el dominio de la vida o de la mue'
Target data:  'l arte de la guerra capitulo 1 sobre la evaluacion sun tzu dice la guerra es de vital importancia para el estado es el dominio de la vida o de la muer'


In [11]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [12]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           9216      
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 36)            36900     
                                                                 
Total params: 5,293,092
Trainable params: 5,293,092
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
checkpoint_dir='/content/gdrive/MyDrive/CheckPoints'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


###P4.2 entrenando

####P4.2a entrenando para usar chekpoints

In [ ]:
EPOCHS=300
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/200


KeyboardInterrupt: ignored


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [15]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model2=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/CheckPoints/cp_0010.ckpt


In [ ]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model2.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history_V2=model2.fit(dataset, 
                    epochs=100, 
                    verbose=1,
                    callbacks=[cp_callback])

AttributeError: ignored

####P4.2b entrenando con tensorboard (opcional)

#####Activando TENSORBOARD 

---
(DASHBOARD para ver el proceso de entrenamiento)

In [ ]:
# You can change the directory name
LOG_DIR = '/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

#####Fit

In [ ]:

#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

##P5. Generando texto nuevo usando la RNN

In [16]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [17]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.3  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [19]:
print(generate_text(model, start_string=u"dominio "))

dominio puer entera de lo piento se comino el entrera de el conto se la cuen se es ente es conter el con es monto anter ecion de la su un entrero en cono conta se ver los conido es ento es entrara lo ser entar entra en puendes de el conde cion ento es la contera si prera el estracion a la es parera con es entra des estera as puerar an an conta lo as ertar con le pue es pue de el concionta es entera conta en la prera de la la conter lo de se parar conto as cuendo es an contar en es conde lo se contara la


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [21]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_arte.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_arte_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_arte_model.h5'))
print("modelo salvado en Drive de google")

FileNotFoundError: ignored

##P7.Cargando un modelo serializado

###P7.1 descargamos el modelo usando wget

####P7.1a descargamos el modelo usando PYRIND & URLLIB (OPCIONAL)

In [ ]:
!pip install pyprind

###P7.2 instanciamos el modelo descargado

In [ ]:

print(generate_text(new_model, start_string=u"colombia es un pais "))